In [69]:
%load_ext autoreload
%autoreload 2

[autoreload of model failed: Traceback (most recent call last):
  File "/home/lvu5/.local/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 261, in check
    superreload(m, reload, self.old_objects)
  File "/home/lvu5/.local/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 484, in superreload
    update_generic(old_obj, new_obj)
  File "/home/lvu5/.local/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 381, in update_generic
    update(a, b)
  File "/home/lvu5/.local/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 333, in update_class
    if update_generic(old_obj, new_obj):
  File "/home/lvu5/.local/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 381, in update_generic
    update(a, b)
  File "/home/lvu5/.local/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 293, in update_function
    setattr(old, name, getattr(new, name))
ValueError: __init__() requires a code object w

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [96]:
import os
import sys
import seaborn as sns
import numpy as np
import pandas as pd
from pandas.core.strings import base
import random
import pickle
import time
from sklearn.utils import shuffle
import warnings
import copy
import math
import argparse
from tqdm import tqdm
from test_nn import predict_nn
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
import lightgbm as lgb
# Fairlearn algorithms and utils
from fairlearn.postprocessing import ThresholdOptimizer
from util import compute_error, get_metrics_df, create_features_dict, find_gamma_superhuman, find_gamma_superhuman_all, load_object, store_object, make_demo_list_filename, make_experiment_filename
from fair_logloss.fair_logloss import DP_fair_logloss_classifier, EOPP_fair_logloss_classifier, EODD_fair_logloss_classifier
from model import LogisticRegression_pytorch, NN_base_superhuman_model, LR_base_superhuman_model

In [97]:
default_args = {'dataset': 'Adult', 'iters': 30, 'num_of_demos': 50, 'num_of_features': 4, 'lr_theta': 0.01, 'noise': 'False', 'noise_ratio': 0.2, 'demo_baseline': 'pp', 'features': ['inacc', 'dp', 'eqodds', 'prp'], 'base_model_type': 'LR', 'num_experiment': 10}
label_dict = {'Adult': 'label', 'COMPAS':'two_year_recid', 'Diabetes': 'label', 'acs_west_poverty': 'POVPIP', 'acs_west_mobility': 'MIG', 'acs_west_income': 'PINCP', 'acs_west_insurance': 'HINS2', 'acs_west_public': 'PUBCOV', 'acs_west_travel': 'JWMNP', 'acs_west_employment': 'ESR'}
protected_dict = {'Adult': 'gender', 'COMPAS':'race',  'Diabetes': 'gender', 'acs_west_poverty': 'RAC1P', 'acs_west_mobility': 'RAC1P', 'acs_west_income': 'RAC1P', 'acs_west_insurance': 'RAC1P', 'acs_west_public': 'RAC1P', 'acs_west_travel': 'RAC1P', 'acs_west_employment': 'RAC1P'}
protected_map = {'Adult': {2:"Female", 1:"Male"}, 'COMPAS': {1:'Caucasian', 0:'African-American'}, \
  'Diabetes': {2:"Female", 1:"Male"},\
  'acs_west_poverty': {0:'White', 1:'Black'}, 'acs_west_mobility': {0:'White', 1:'Black'},\
  'acs_west_income': {0:'White', 1:'Black'}, 'acs_west_insurance': {0:'White', 1:'Black'},\
  'acs_west_public': {0:'White', 1:'Black'}, 'acs_west_travel': {0:'White', 1:'Black'}, 'acs_west_employment': {0:'White', 1:'Black'}}

lr_theta = default_args['lr_theta']
num_of_demos = default_args['num_of_demos']
noise_ratio = default_args['noise_ratio']
iters = default_args['iters']
alpha = 0.5
beta = 0.5
lamda = 0.001
#demo_baseline = "pp" #"fair_logloss" 
model = "logistic_regression"
noise_list = [0.2]#0.03, 0.04]#[0.06, 0.07, 0.08, 0.09]##[0.16, 0.17, 0.18, 0.19, 0.20]#[0.11, 0.12, 0.13, 0.14, 0.15]



In [181]:
class Super_human:

  def __init__(self, dataset, num_of_demos, feature, num_of_features, lr_theta, noise, noise_ratio, demo_baseline, base_model_type):
    self.dataset = dataset
    self.num_of_demos = num_of_demos
    self.num_of_features = num_of_features
    self.feature = feature
    self.alpha = [1.0 for _ in range(self.num_of_features)]
    self.gamma_superhuman = [0.0 for _ in range(self.num_of_features)]
    self.label = label_dict[dataset] ##
    print("self.label: ", self.label)
    self.sensitive_feature = protected_dict[dataset] ##
    self.dict_map = protected_map[dataset] ##
    self.lamda = lamda
    self.c = None
    self.lr_theta = lr_theta
    self.noise_ratio = noise_ratio
    self.noise = noise
    self.base_model_type = base_model_type
    self.demo_baseline = demo_baseline
    self.set_paths()
    self.dataset_ref = pd.read_csv(self.dataset_path, index_col=0) #self.dataset_ref = pd.read_csv('dataset_ref.csv', index_col=0)
    self.num_of_attributs = self.dataset_ref.shape[1] - 1 # discard self.label
    self.model_params = None
    self.logi_params = {
        'C': 100,
        'penalty': 'l2',
        'solver': 'newton-cg',
        'max_iter': 1000
    }

  class data_demo:
    def __init__(self, train_x=None, test_x=None, train_y=None, test_y=None, train_A=None, test_A=None, train_A_str=None, test_A_str=None ,idx_train=None, idx_test=None):
        self.train_x = train_x
        self.test_x = test_x
        self.train_y = train_y
        self.test_y = test_y
        self.train_A = train_A
        self.test_A = test_A
        self.train_A_str = train_A_str
        self.test_A_str = test_A_str
        self.idx_train = idx_train
        self.idx_test = idx_test
        self.metric = {}
  def read_demo_list(self):
    file_dir = os.path.join(self.data_path)
    print("file_dir in read_demo_list: ", file_dir)
    demo_list_filename = make_demo_list_filename(dataset = self.dataset, demo_baseline = self.demo_baseline, num_of_demos = self.num_of_demos, noise_ratio = self.noise_ratio)
    print("demo_list file name: ")
    print(demo_list_filename)
    self.demo_list = load_object(file_dir, demo_list_filename, -1)
    return self.demo_list
  
  def set_paths(self):
    if self.noise:
      root = "experiments/noise"
    else:
      root = "experiments"
    print("root: ", root)
    self.data_path = os.path.join(root,"data")
    self.model_path = os.path.join(root,"model")
    self.train_data_path = os.path.join(root, "train")
    self.test_data_path = os.path.join(root, "test")
    self.plots_path = os.path.join(root,"plots")
    self.dataset_path = os.path.join("dataset", self.dataset, "dataset_ref.csv")
    
  def base_model(self):
    self.model_name = "logistic-regression"
    train_data_filename = "train_data_" + make_experiment_filename(dataset = self.dataset, demo_baseline = self.demo_baseline, lr_theta = self.lr_theta, num_of_demos = self.num_of_demos, noise_ratio = self.noise_ratio) + ".csv"
    train_file_path = os.path.join(self.train_data_path, train_data_filename)
    self.train_data = pd.read_csv(train_file_path, index_col=0)

    A = self.train_data[self.sensitive_feature]
    A_str = A.map(self.dict_map)
    # Extract the target
    Y = self.train_data[self.label]
    X = self.train_data.drop(columns=[self.label])
    
    X_train, X_test, Y_train, Y_test, A_train, A_test, A_str_train, A_str_test = train_test_split(
        X,
        Y,
        A,
        A_str,
        test_size = 1 - alpha,
        random_state=12345,
        stratify=Y
        )

    self.model_obj = LogisticRegression_pytorch(X_train.shape[1], 2)
    print("self.model_obj.type: ", self.model_obj.type)
    self.model_obj.fit(X_train, Y_train)
    self.pred_scores = self.model_obj.predict_proba(X_test)
    dataset = self.dataset
    if self.dataset == 'COMPAS':
      mode = 'demographic_parity' #'equalized_opportunity' # #'equalized_odds'
      C = .005
      if mode == 'demographic_parity':
        h = DP_fair_logloss_classifier(C=C, random_initialization=True, verbose=False)
      elif mode == 'equalized_opportunity':
        h = EOPP_fair_logloss_classifier(C=C, random_initialization=True, verbose=False)
      elif mode == 'equalized_odds':
        h = EODD_fair_logloss_classifier(C=C, random_initialization=True, verbose=False)    
      else:
        raise ValueError('Invalid second arg')
    
      Y_train = Y_train.astype('float64')
      Y_test = Y_test.astype('float64')
      A_train = A_train.astype('float64')
      A_test = A_test.astype('float64')
      if self.dataset == 'Adult':
        A_train = A_train - 1
        A_test = A_test - 1

      for c in list(X_train.columns):
          if X_train[c].min() < 0 or X_train[c].max() > 1:
              mu = X_train[c].mean()
              s = X_train[c].std(ddof=0)
              X_train.loc[:,c] = (X_train[c] - mu) / s
              X_train.loc[:,c] = (X_train[c] - mu) / s
      
      h.fit(X_train.values,Y_train.values,A_train.values)
      self.pred_scores = h.predict(X_test.values,A_test.values)
      
      print("h.theta: ", h.theta)
      self.model_obj.update_model_theta(h.theta[:-1])
      #self.model_obj.coef_ = np.asarray([h.theta[:-1]])
      print("self.model_obj.coef_: ", self.model_obj.get_model_theta())

    self.base_dict = {"model_obj": self.model_obj,
                 "pred_scores": self.pred_scores,
                 "model_name": self.model_name,
                 "logi_params": self.logi_params,
                 "X_train": X_train,
                 "Y_train": Y_train,
                 "X_test": X_test,
                 "Y_test": Y_test,
                 "A_str_train": A_str_train,
                 "A_str_test": A_str_test}
    
    model_file_dir = os.path.join(self.model_path, 'base_model_' + dataset + '.pickle') 
    with open(model_file_dir, 'wb') as handle:  #with open(f'base_model_{dataset}.pickle', 'wb') as handle:
        pickle.dump(self.base_dict, handle)



In [182]:
%reload_ext autoreload

In [183]:
from model import LogisticRegression_pytorch

In [184]:
sh_obj = Super_human(dataset = 'Adult', num_of_demos = 50, feature = ['inacc', 'dp', 'eqodds', 'prp'], num_of_features = 4, lr_theta = lr_theta, noise = False, noise_ratio = 0.0, demo_baseline= 'pp', base_model_type = 'LR')

self.label:  label
root:  experiments


In [185]:
sh_obj.base_model()

self.model_obj.type:  LR_pytorch
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'>
heeloooo
<class 'pandas.core.frame.DataFrame'>
tensor([0., 0., 0.,  ..., 1., 0., 0.])
outputs:  tensor([[0.4653, 0.5182],
        [0.4835, 0.5628],
        [0.4820, 0.5034],
        ...,
        [0.4831, 0.4424],
        [0.4452, 0.5623],
        [0.5325, 0.5369]], grad_fn=<SigmoidBackward0>)
outputs:  tensor([[0.4654, 0.5181],
        [0.4836, 0.5627],
        [0.4821, 0.5033],
        ...,
        [0.4831, 0.4424],
        [0.4454, 0.5622],
        [0.5327, 0.5367]], grad_fn=<SigmoidBackward0>)
outputs:  tensor([[0.4655, 0.5180],
        [0.4837, 0.5626],
        [0.4822, 0.5031],
        ...,
        [0.4832, 0.4423],
        [0.4455, 0.5620],
        [0.5328, 0.5366]], grad_fn=<SigmoidBackward0>)
outputs:  tensor([[0.4656, 0.5179],
        [0.4837, 0.5625],
        [0.4824, 0.5030],
        ...,
        [0.4832, 0.4423],
        [0.4456, 0.5619],
        [0.5330, 0.5364]], gra

In [186]:
sh_obj.read_demo_list()

file_dir in read_demo_list:  experiments/data
demo_list file name: 
demo_list_Adult_pp_50_0-0


In [ ]:
sh_obj.update_model(lr_theta, iters)